## Cleaning US Dataset

In [1]:
# Standard libraries
from pathlib import Path

# Third-party libraries
import numpy as np
import pandas as pd


def main():
    # Setting US_df as global to use across code
    global US_df

    # Defining file name and path, and reading the CSV
    file_name = "final_US_quotes.csv"
    output_path = Path.cwd().parent / "data" / "raw" / file_name  # Obtaining the directory
    US_df = pd.read_csv(output_path)

    # Removing " Year Term" from the "Term Length" column and converting to integer
    US_df["Term Length"] = US_df["Term Length"].str.replace(" Year Term", "").astype(int)

    # Removing commas and "$" sign, then convert to float
    US_df["Premium"] = US_df["Premium"].replace({r"\$": "", ",": ""}, regex=True).astype(float)

    # Defining exchange rate (retrieved at time finished collecting final data)
    exchange_rate = 0.74616745 # USD to GBP, source: (XE (n.d.), retrieved on 21-04-2025)

    # Overwriting the Premium and coverage amount column with GBP values
    US_df["Premium"] = (US_df["Premium"] * exchange_rate).round(2)
    US_df["Coverage Amount"] = (US_df["Coverage Amount"] * exchange_rate).round(2)
    US_df.rename(columns={"Premium": "Premium (£)"}, inplace=True)

    # Adding "Country" column to differentiate after merging
    US_df["Country"] = "US"


    # Printing the DataFrame
    display(US_df)


main()


,Coverage Amount,Term Length,Age,Gender,Is_Smoker,Premium (£),Country
0,74616.74,10,20,Male,Current user,16.17,US
1,74616.74,10,20,Male,Current user,16.65,US
2,74616.74,10,20,Male,Current user,17.56,US
3,74616.74,10,20,Male,Current user,19.04,US
4,74616.74,10,20,Male,Current user,20.79,US
...,...,...,...,...,...,...,...
2281,1492334.90,30,50,Male,Never Used,507.15,US
2282,1492334.90,30,50,Female,Current user,1017.22,US
2283,1492334.90,30,50,Female,Current user,1687.08,US
2284,1492334.90,30,50,Female,Never Used,305.65,US


XE.com. (n.d.). XE Currency Converter. Retrieved April 21st, 2025,<br>from <https://www.xe.com/currencyconverter/convert/?Amount=1&From=USD&To=GBP>

## Cleaning UK Dataset

In [2]:
def main():
    # Setting UK_df as global to use across code
    global UK_df

    # Defining file name and path, and reading the CSV
    input_file = "final_UK_quotes.csv"
    output_path = Path.cwd().parent / "data" / "raw" / input_file  # Obtaining the directory
    UK_df = pd.read_csv(output_path)

    # Removing commas and "$" sign, then convert to float
    UK_df["Premium"] = UK_df["Premium"].replace({r"\£": "", ",": ""}, regex=True).astype(float)
    UK_df.rename(columns={"Premium": "Premium (£)"}, inplace=True) 

    # Adding "Country" column to differentiate after merging
    UK_df["Country"] = "UK"


    # Printing the DataFrame
    display(UK_df)
    

main()

,Coverage Amount,Term Length,Age,Gender,Is_Smoker,Premium (£),Country
0,100000,10,20,Male,Smoker,5.00,UK
1,100000,10,20,Male,Smoker,5.01,UK
2,100000,10,20,Male,Smoker,5.01,UK
3,100000,10,20,Male,Smoker,5.18,UK
4,100000,10,20,Male,Smoker,5.48,UK
...,...,...,...,...,...,...,...
11889,350000,25,65,Male,Smoker,623.46,UK
11890,350000,25,65,Male,Smoker,640.32,UK
11891,350000,25,65,Male,Smoker,645.73,UK
11892,350000,25,65,Male,Smoker,711.16,UK


## Merging US and UK Datasets

In [3]:
def main():
    # Clean column names in both DataFrames
    UK_df.columns = UK_df.columns.str.strip()
    US_df.columns = US_df.columns.str.strip()

    quotes_df = pd.concat([US_df, UK_df], axis=0, ignore_index=True)

    file_name = "all_quotes.csv"
    # Define the file path (adjust the path as needed)
    output_path = Path.cwd().parent / "data" / "clean" / file_name
    
    # Renaming 'Term Length to 'Term_Length for more consistent column names
    quotes_df.rename(columns={"Term Length": "Term_Length"}, inplace=True)

    # Renaming "Gender" to "Is_Male" and changing the values to 1/0 based on gender
    quotes_df.rename(columns={"Gender": "Is_Male"}, inplace=True)
    quotes_df["Is_Male"] = quotes_df["Is_Male"].apply(lambda x: 1 if x == "Male" else 0)

    # Changing the values in "Is_Smoker" column to 1/0 based on smoking status
    quotes_df["Is_Smoker"] = quotes_df["Is_Smoker"].apply(lambda x: 1 if x in ["Current user", "Smoker"] else 0)
    
    # Renaming "Country" to "Is_UK" and changing to binary variable
    quotes_df.rename(columns={"Country": "Is_UK"}, inplace=True)
    quotes_df["Is_UK"] = quotes_df["Is_UK"].apply(lambda x: 1 if x == "UK" else 0)

    # Log-transforming coverage amount
    quotes_df["ln(Coverage_Amount)"] = np.log(quotes_df["Coverage Amount"])
    quotes_df.drop(columns="Coverage Amount", inplace=True)
    
    # Log-transforming premiums (raw premiums will still remain for later use)
    quotes_df["ln(Premium)"] = np.log(quotes_df["Premium (£)"])

    # Reorder columns
    quotes_df = quotes_df[["ln(Coverage_Amount)", "Term_Length", "Age", "Is_Male", "Is_Smoker", "Is_UK", "Premium (£)", "ln(Premium)"]]
    
    # Loop through each categorical variable and print value counts (for table in final blog)
    for col in ["Is_Male", "Is_Smoker", "Is_UK"]:
        print(f"--- {col} ---")
        print(f"{quotes_df[col].value_counts(dropna=False)}\n")

    
    
    # Saving DataFrame to CSV
    quotes_df.to_csv(output_path, index=False)
    print(f"📁 All done — saved {file_name} file to {output_path}")


main()

--- Is_Male ---
Is_Male
1    7133
0    7047
Name: count, dtype: int64

--- Is_Smoker ---
Is_Smoker
0    7102
1    7078
Name: count, dtype: int64

--- Is_UK ---
Is_UK
1    11894
0     2286
Name: count, dtype: int64

📁 All done — saved all_quotes.csv file to /home/freit/code-projects/life-insurance-scraper-ml-study/data/clean/all_quotes.csv
